# Semantische Transformation

Du kennst jetzt unterschiedliche Word Embeddings und kannst damit Ähnlichkeiten zwischen Wörtern selbst bestimmen.

Allerdings gehen die Möglichkeiten der Embeddings noch weiter darüber hinaus. So kannst du eine Kombination von TF/IDF und Embeddings nutzen, um Texte in einen "semantischen Raum" zu transformieren und dort auch Ähnlichkeiten von Wörtern (oder Konzepten) ermitteln sowie semantische Suchen durchführen.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [1]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [2]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Daten vektorisieren

Damit du  häufige Wörter niedriger gewichten kannst, berechnest du zunächst die TF/IDF-Vektoren der Dokumente:

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words

tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=10, sublinear_tf=True, use_idf=False)
tfidf_vectors = tfidf_vectorizer.fit_transform(df["nav"])

## Wortvektoren berechnen (bzw. einladen)

Anschließend lädst du die bereits berechneten Word Embeddings:

In [4]:
if ON_COLAB:
    os.system("test -f heise-articles-2020.w2v || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.w2v.gz && gunzip heise-articles-2020.w2v.gz")
    w2v_file = "heise-articles-2020.w2v"
else:
    w2v_file = "../99_Common/heise-articles-2020.w2v"

In [ ]:
!pip install "gensim>=4.0.0"

In [5]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format(w2v_file)

/home/cwinkler/.venv/jupyter-new/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Stattdessen könntest du auch ein fertiges Embedding nutzen, wenn deine Datenmenge zu klein ist.

## Semantische Tranformation durchführen

Die Funktion zur semantischen Transformation ist etwas komplizierter.

* Du beginnst mit einer leeren Liste gemittelter Dokumentvektoren
* Du iterierst zunächst über alle TF/IDF-Vektoren (`shape[0]` gibt dir die Anzahl der Zeilen), also über alle Dokumente
* Du setzt den gemittelten Wortvektor des aktuellen Dokuments auf 0
* Anschließend lässt du dir die Spalten ausgeben, die Werte != 0 haben (das entsprechende Wort kommt im Dokument vor)
* Wenn sich das Wort auch im Embedding-Vokabular befindet, addierst du den (normierten!) Wortvektor gewichtet mit dem TF/IDF-Maß
* Falls sich ein gemittelter Wortvektor ungleich des Nullvektors ergeben hat, wird er normiert
* Anschließend fügst du den Wortvektor in die Liste der gemittelten Dokumentvektoren an

In [6]:
from tqdm import tqdm
import numpy as np

# averaged word vectors
awv = []
fn = tfidf_vectorizer.get_feature_names()
for i in tqdm(range(tfidf_vectors.shape[0])):
    v = np.zeros(w2v.vector_size)
    # immer nur eine Zeile, rows wird nicht benötigt
    rows,cols = tfidf_vectors[i].nonzero()
    for c in cols:
        feature = fn[c]
        if feature in w2v.key_to_index :
            # TF/IDF als Gewicht des normierten Wortvektors
            wv = w2v[feature]
            v += tfidf_vectors[i][0, c] * wv / np.linalg.norm(wv)
    if np.linalg.norm(v) > 0:
        v = v/np.linalg.norm(v)
    awv.append(v)

100%|██████████| 10403/10403 [01:44<00:00, 99.25it/s] 


Nun kannst du Ähnlichkeiten berechnen, dazu nutzt du die dir schon bekannte Methode `cosine_similarity`:

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

Möchtest du z.B. die Dokumente finden, die die größte *semantische Ähnlichkeit* zu `apple` haben, kannst du zunächst den Vektor bestimmen:

In [8]:
apple = w2v["apple"]

Anschließend berechnest du den Cosinus-Abstand zu allen Dokumenten (die Funktion erwartet zwei Matrizen, daher ist der zweite Vektor in `[]`):

In [9]:
r = cosine_similarity(awv, [apple])

Dich interessiert nur das Dokument mit der größten Ähnlichkeit:

In [10]:
df.iloc[r.argmax()]

title             Apple TV+ und iTunes künftig auch auf LG-Ferns...
header            Die Apple-TV-App soll nicht nur für Fernseher ...
author                                                  Nico Jurran
text              LG hat bekanntgegeben, die Apple-TV-App auf se...
keywords                                Apple TV, CES, CES 2020, LG
url               https://www.heise.de/newsticker/meldung/Apple-...
datePublished                                   2020-01-07 08:03:00
commentCount                                                     16
full_text         Apple TV+ und iTunes künftig auch auf LG-Ferns...
nouns             Apple|TV+|iTunes|LG-Fernsehern#Apple-TV-App|Fe...
adjectives        künftig|auch#nur|auch#erst|darüber|direkt|zwar...
verbs             soll|kommen|werden|nachrüsten#haben|bekanntgeb...
lemmas            Apple|TV+|und|iTunes|künftig|auch|auf|LG-Ferns...
nav               Apple|TV+|iTunes|künftig|auch|LG-Fernsehern#Ap...
entities          Apple TV+|iTunes|LG-Fernsehern

Spannend wird es jetzt, wenn du die Wortvektoren unterschiedlicher Wörter kombinierst. Da die Wortvektoren nicht normiert sind, musst du das noch nachholen, weil sonst verschiedene Wörter unterschiedlich stark beitragen würden:

In [11]:
wv =  w2v["google"]/np.linalg.norm(w2v["google"])
wv += w2v["umsatz"]/np.linalg.norm(w2v["umsatz"])
r = cosine_similarity(awv, [wv])
df.iloc[r.argmax()]

title             Einnahmeschätzung: App-Store-Umsatz fast doppe...
header            Apples Anwendungsladen soll im dritten Quartal...
author                                                   Ben Schwan
text              Apple macht mit seinem App Store weiterhin her...
keywords          Android, App Store, Apple, Apps, Einnahmen, Ge...
url               https://www.heise.de/news/Einnahmeschaetzung-A...
datePublished                             2020-10-05 17:48:00+02:00
commentCount                                                    102
full_text         Einnahmeschätzung: App-Store-Umsatz fast doppe...
nouns             Einnahmeschätzung#App-Store-Umsatz|Google|Play...
adjectives        fast|doppeln|so|hoch#dritt|erneut|fett#langsam...
verbs             soll|schreiben|haben#wachsen#machen#sein#hochr...
lemmas            Einnahmeschätzung|:#App-Store-Umsatz|fast|dopp...
nav               Einnahmeschätzung#App-Store-Umsatz|fast|doppel...
entities          App-Store-Umsatz|Google Play#A

In [12]:
wv =  w2v["google"]/np.linalg.norm(w2v["google"])
wv += w2v["umsatz"]/np.linalg.norm(w2v["umsatz"])
wv += w2v["microsoft"]/np.linalg.norm(w2v["microsoft"])
r = cosine_similarity(awv, [wv])
df.iloc[r.argmax()]

title                     Microsoft schließt Streaming-Portal Mixer
header            Mit Mixer wollte Microsoft dem Amazon-Portal T...
author                                                Daniel Herbig
text              Microsofts Streaming-Portal Mixer schließt am ...
keywords          Microsoft, Mixer, Spiele, Streaming-Dienst, Tw...
url               https://www.heise.de/news/Microsoft-schliesst-...
datePublished                             2020-06-23 09:29:00+02:00
commentCount                                                     35
full_text         Microsoft schließt Streaming-Portal Mixer\nMit...
nouns             Microsoft|Streaming-Portal#Mixer#Mixer|Microso...
adjectives        Nun#22.#Auch|dort|weiterhin#Allerdings|nun|auc...
verbs             schließen#wollen|bereit#soll|schließen|werden#...
lemmas            Microsoft|schließen|Streaming-Portal#Mixer#\n#...
nav               Microsoft|schließen|Streaming-Portal#Mixer#Mix...
entities          Microsoft|Streaming-Portal#Mic

In [13]:
print(df.iloc[r.argmax()]["full_text"])

Microsoft schließt Streaming-Portal Mixer
Mit Mixer wollte Microsoft dem Amazon-Portal Twitch Konkurrenz bereiten. Nun soll das Streaming-Portal geschlossen werden. Nutzer werden zu Facebook geschickt.
Microsofts Streaming-Portal Mixer schließt am 22. Juli. Das hat Microsoft angekündigt. Nutzer des Streaming-Dienstes sollen nach dem Mixer-Aus zu Facebooks Alternative Facebook Gaming weitergeleitet werden.
Auch die Mixer-Streamer sollen zu Facebook Gaming wechseln und dort weiterstreamen – Verträge sollen nach Möglichkeit weiterhin gelten. Allerdings haben die Streamer nun auch die Möglichkeit, davon zurückzutreten und zu anderen Plattformen zu gehen. Mixer-Nutzer, die noch Abonnements und Währungen für das Microsoft-Portal besitzen, sollen mit Xbox-Geschenkkarten entschädigt werden, schreibt Microsoft in einem Blog-Eintrag.
Als Streaming-Portal mit Fokus auf Videospiele sollte Mixer vor allem Amazons Marktführer Twitch Konkurrenz bereiten. Das hat sich Microsoft viel kosten lassen: Bei

## Semantische Methoden durch Kombination

Du hast gesehen, wie sich Word Embeddings mit "traditionellen" Machine Learning-Methoden kombinieren lassen.

Das Ergebnis ist durchaus beeindruckend. Besonders im letzten Fall wird eine Meldung gefunden, in der das Wort "Umsatz" gar nicht vorkommt, die sich aber dennoch damit beschäftigt.